Based on code by RCubo - https://github.com/RCubo/Titanic-Keras/blob/master/titanic_keras.py

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jan 21 14:07:17 2017
@author: Ruben
"""

import numpy as num;
import scipy as sci;
import csv;
import matplotlib.pyplot as plt;
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils


def dataInit():
    #Reads the .csv files in order to initialize the data
    
    #For the train set
    fop = open(r'train.csv',encoding = 'utf-8')
    
    trainReader = csv.reader(fop, delimiter=',', quotechar='"') #To take out the header
    
    trainHeader = trainReader.__next__(); 
    
    print(trainHeader)
    
    eofFlag = False #Flag for end of file
    
    trainInfo = [];
    
    while (eofFlag == False):
        try:
            trainInfo.append(trainReader.__next__());
        except StopIteration:         #Will be thrown at the end of the file
            eofFlag = True;
            
    fop.close();   
        
    #Same procedure for the test set
    fop = open(r'test.csv',encoding = 'utf-8')
    
    testReader = csv.reader(fop, delimiter=',', quotechar='"')
    
    testHeader = testReader.__next__();
    
    print(testHeader)
    
    eofFlag = False
    
    testInfo = [];
    
    while (eofFlag == False):
        try:
            testInfo.append(testReader.__next__());
        except StopIteration:        
            eofFlag = True;
            
    fop.close();   
    
    return trainInfo, testInfo

def dfPreProcessingTrain(trainInfo):
    #Processes a list given by dataInit into a DataFrame, converting the
    #data into suitable types (namely numeric)
    
    dfTrainInfo = pd.DataFrame(trainInfo);
    
    #Assign column names for improved readbility                          
    dfTrainInfo.columns = ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
    
    #Convert everything that can be converted to numeric.
    dfTrainInfo['Survived']=num.round(pd.to_numeric(dfTrainInfo['Survived'],errors='coerce'))
    dfTrainInfo['Pclass']=num.round(pd.to_numeric(dfTrainInfo['Pclass'],errors='coerce'))
    dfTrainInfo['Age']=num.round(pd.to_numeric(dfTrainInfo['Age'],errors='coerce'))
    dfTrainInfo['SibSp']=pd.to_numeric(dfTrainInfo['SibSp'],errors='coerce')
    dfTrainInfo['Parch']=pd.to_numeric(dfTrainInfo['Parch'],errors='coerce')
    return dfTrainInfo

def dfPreProcessingTest(testInfo):
    #Same as dfPreProcessingTrain but for the test set
    #The main difference is that the TestSet does not have survival data.
    
    dfTestInfo = pd.DataFrame(testInfo);
    dfTestInfo.columns = ['PassengerId','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
    dfTestInfo['Pclass']=num.round(pd.to_numeric(dfTestInfo['Pclass'],errors='coerce'))
    dfTestInfo['Age']=num.round(pd.to_numeric(dfTestInfo['Age'],errors='coerce'))
    dfTestInfo['SibSp']=pd.to_numeric(dfTestInfo['SibSp'],errors='coerce')
    dfTestInfo['Parch']=pd.to_numeric(dfTestInfo['Parch'],errors='coerce')
    return dfTestInfo

def assignSex(dfTrainInfo):
    #Assign a number for each sex. 1 if the passenger is male and 2 if it's female
    
    #Divide the dataframe into two
    dfMales = dfTrainInfo.loc[dfTrainInfo['Sex']=='male']
    dfFemales = dfTrainInfo.loc[dfTrainInfo['Sex']=='female']
    
    #Substitute the value in the Sex column
    dfMales['Sex'] = 1
    dfFemales['Sex'] = 2
    
    #Put everything together into another DataFrame (Probably there are better ways to do this...)
    frames = [dfMales,dfFemales]
    dfSexDivided = pd.concat(frames)
    dfSexDivided = dfSexDivided.sort_index()
    
    return dfSexDivided

#Main program

#Read the .csv files
trainInfo, testInfo = dataInit();
                        
#Convert them to dataframes
dfTrainInfo = dfPreProcessingTrain(trainInfo);
dfTestInfo = dfPreProcessingTest(testInfo);  

#For the train set:

#Convert Sex to a number                                
dfTrainInfoSex = assignSex(dfTrainInfo);

#Select features for the ANN                                
#dfTrainFeatures = dfTrainInfoSex[['Pclass','Sex','Age','SibSp','Parch']]  
dfTrainFeatures = dfTrainInfoSex[['Pclass','Sex','Parch']]

#Select the label. In this case the labels are survival or not  
dfTrainLabels = dfTrainInfoSex['Survived']

#Convert to array for processing
arrayTrainFeatures = num.array(dfTrainFeatures)
arrayTrainLabels = num.array(dfTrainLabels)


#Same with the test set (no labels were given, only features)

dfTestInfoSex = assignSex(dfTestInfo);
                         
#dfTestFeatures = dfTestInfoSex[['Pclass','Sex','Age','SibSp','Parch']]  
dfTestFeatures = dfTestInfoSex[['Pclass','Sex','Parch']]  

arrayTestFeatures = num.array(dfTestFeatures)

#ANN Properties

nb_epoch = 2000 #Probably too high, but it works
batch_size = 32

#Assign an average value if an age is not known (they only have some NaNs in Ages)
arrayTrainFeatures[num.isnan(arrayTrainFeatures)] = 29
arrayTestFeatures[num.isnan(arrayTestFeatures)] = 29
                  
#Define the ANN
annModel = Sequential() #Sequential ANN

#Layers of the ANN (Dense in this case)
annModel.add(Dense(30, input_dim=3, activation='sigmoid'))
annModel.add(Dense(3, input_dim=30, activation='sigmoid'))
annModel.add(Dense(1, input_dim=3, activation='sigmoid'))
annModel.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#Get the ANN parameters                   
annModel.fit(arrayTrainFeatures, arrayTrainLabels,
              validation_split=0.2,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              shuffle=True)

#Predict survival (taking 0.5 as the cutoff)
TestPred = num.round(annModel.predict(arrayTestFeatures))

#Copying everything to a DataFrame
dfSurvived = dfTestInfo;
dfSurvived['Survived'] = TestPred.astype(int);          

#Finally write the DataFrame to a .csv file          
dfSurvived.to_csv(path_or_buf='r_cubo_result.csv',index=False,columns=['PassengerId','Survived'])

Using TensorFlow backend.


['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


/home/ikanez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ikanez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ikanez/anaconda3/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 712 samples, validate on 179 samples
Epoch 1/2000
712/712 [==============================] - 0s - loss: 0.6777 - acc: 0.6096 - val_loss: 0.6676 - val_acc: 0.6425
Epoch 2/2000
712/712 [==============================] - 0s - loss: 0.6730 - acc: 0.6096 - val_loss: 0.6620 - val_acc: 0.6425
Epoch 3/2000
712/712 [==============================] - 0s - loss: 0.6700 - acc: 0.6096 - val_loss: 0.6583 - val_acc: 0.6425
Epoch 4/2000
712/712 [==============================] - 0s - loss: 0.6679 - acc: 0.6096 - val_loss: 0.6546 - val_acc: 0.6425
Epoch 5/2000
712/712 [==============================] - 0s - loss: 0.6663 - acc: 0.6096 - val_loss: 0.6522 - val_acc: 0.6425
Epoch 6/2000
712/712 [==============================] - 0s - loss: 0.6651 - acc: 0.6096 - val_loss: 0.6509 - val_acc: 0.6425
Epoch 7/2000
712/712 [==============================] - 0s - loss: 0.6643 - acc: 0.6096 - val_loss: 0.6492 - val_acc: 0.6425
Epoch 8/2000
712/712 [==============================] - 0s - loss: 0.6633 - acc

712/712 [==============================] - 0s - loss: 0.5227 - acc: 0.7683 - val_loss: 0.4868 - val_acc: 0.8156
Epoch 66/2000
712/712 [==============================] - 0s - loss: 0.5210 - acc: 0.7683 - val_loss: 0.4850 - val_acc: 0.8212
Epoch 67/2000
712/712 [==============================] - 0s - loss: 0.5190 - acc: 0.7711 - val_loss: 0.4825 - val_acc: 0.8156
Epoch 68/2000
712/712 [==============================] - 0s - loss: 0.5171 - acc: 0.7725 - val_loss: 0.4805 - val_acc: 0.8156
Epoch 69/2000
712/712 [==============================] - 0s - loss: 0.5157 - acc: 0.7795 - val_loss: 0.4799 - val_acc: 0.8156
Epoch 70/2000
712/712 [==============================] - 0s - loss: 0.5137 - acc: 0.7500 - val_loss: 0.4785 - val_acc: 0.8156
Epoch 71/2000
712/712 [==============================] - 0s - loss: 0.5124 - acc: 0.7683 - val_loss: 0.4764 - val_acc: 0.8156
Epoch 72/2000
712/712 [==============================] - 0s - loss: 0.5111 - acc: 0.7781 - val_loss: 0.4748 - val_acc: 0.8156
Epoch 

...

712/712 [==============================] - 0s - loss: 0.4513 - acc: 0.7935 - val_loss: 0.4157 - val_acc: 0.8268
Epoch 1729/2000
712/712 [==============================] - 0s - loss: 0.4515 - acc: 0.7949 - val_loss: 0.4148 - val_acc: 0.8268
Epoch 1730/2000
712/712 [==============================] - 0s - loss: 0.4522 - acc: 0.7949 - val_loss: 0.4156 - val_acc: 0.8268
Epoch 1731/2000
712/712 [==============================] - 0s - loss: 0.4518 - acc: 0.7978 - val_loss: 0.4151 - val_acc: 0.8268
Epoch 1732/2000
712/712 [==============================] - 0s - loss: 0.4514 - acc: 0.7992 - val_loss: 0.4157 - val_acc: 0.8268
Epoch 1733/2000
712/712 [==============================] - 0s - loss: 0.4516 - acc: 0.7935 - val_loss: 0.4146 - val_acc: 0.8268
Epoch 1734/2000
712/712 [==============================] - 0s - loss: 0.4515 - acc: 0.7963 - val_loss: 0.4152 - val_acc: 0.8268
Epoch 1735/2000
712/712 [==============================] - 0s - loss: 0.4517 - acc: 0.7935 - val_loss: 0.4150 - val_acc:

712/712 [==============================] - 0s - loss: 0.4518 - acc: 0.7992 - val_loss: 0.4149 - val_acc: 0.8268
Epoch 1793/2000
712/712 [==============================] - 0s - loss: 0.4519 - acc: 0.7893 - val_loss: 0.4152 - val_acc: 0.8268
Epoch 1794/2000
712/712 [==============================] - 0s - loss: 0.4520 - acc: 0.7963 - val_loss: 0.4152 - val_acc: 0.8268
Epoch 1795/2000
712/712 [==============================] - 0s - loss: 0.4512 - acc: 0.7963 - val_loss: 0.4147 - val_acc: 0.8268
Epoch 1796/2000
712/712 [==============================] - 0s - loss: 0.4519 - acc: 0.7893 - val_loss: 0.4151 - val_acc: 0.8268
Epoch 1797/2000
712/712 [==============================] - 0s - loss: 0.4518 - acc: 0.7949 - val_loss: 0.4148 - val_acc: 0.8268
Epoch 1798/2000
712/712 [==============================] - 0s - loss: 0.4520 - acc: 0.7963 - val_loss: 0.4145 - val_acc: 0.8268
Epoch 1799/2000
712/712 [==============================] - 0s - loss: 0.4514 - acc: 0.7949 - val_loss: 0.4160 - val_acc:

712/712 [==============================] - 0s - loss: 0.4517 - acc: 0.7949 - val_loss: 0.4154 - val_acc: 0.8268
Epoch 1857/2000
712/712 [==============================] - 0s - loss: 0.4520 - acc: 0.7921 - val_loss: 0.4155 - val_acc: 0.8268
Epoch 1858/2000
712/712 [==============================] - 0s - loss: 0.4515 - acc: 0.7963 - val_loss: 0.4154 - val_acc: 0.8268
Epoch 1859/2000
712/712 [==============================] - 0s - loss: 0.4518 - acc: 0.7949 - val_loss: 0.4157 - val_acc: 0.8268
Epoch 1860/2000
712/712 [==============================] - 0s - loss: 0.4516 - acc: 0.7963 - val_loss: 0.4147 - val_acc: 0.8268
Epoch 1861/2000
712/712 [==============================] - 0s - loss: 0.4518 - acc: 0.7935 - val_loss: 0.4147 - val_acc: 0.8268
Epoch 1862/2000
712/712 [==============================] - 0s - loss: 0.4516 - acc: 0.7963 - val_loss: 0.4145 - val_acc: 0.8268
Epoch 1863/2000
712/712 [==============================] - 0s - loss: 0.4516 - acc: 0.7963 - val_loss: 0.4147 - val_acc:

712/712 [==============================] - 0s - loss: 0.4526 - acc: 0.7963 - val_loss: 0.4151 - val_acc: 0.8268
Epoch 1921/2000
712/712 [==============================] - 0s - loss: 0.4515 - acc: 0.7963 - val_loss: 0.4145 - val_acc: 0.8268
Epoch 1922/2000
712/712 [==============================] - 0s - loss: 0.4518 - acc: 0.7921 - val_loss: 0.4153 - val_acc: 0.8268
Epoch 1923/2000
712/712 [==============================] - 0s - loss: 0.4516 - acc: 0.7949 - val_loss: 0.4156 - val_acc: 0.8268
Epoch 1924/2000
712/712 [==============================] - 0s - loss: 0.4518 - acc: 0.7963 - val_loss: 0.4150 - val_acc: 0.8268
Epoch 1925/2000
712/712 [==============================] - 0s - loss: 0.4517 - acc: 0.7921 - val_loss: 0.4168 - val_acc: 0.8268
Epoch 1926/2000
712/712 [==============================] - 0s - loss: 0.4519 - acc: 0.7963 - val_loss: 0.4154 - val_acc: 0.8268
Epoch 1927/2000
712/712 [==============================] - 0s - loss: 0.4517 - acc: 0.7978 - val_loss: 0.4146 - val_acc:

712/712 [==============================] - 0s - loss: 0.4516 - acc: 0.7935 - val_loss: 0.4145 - val_acc: 0.8268
Epoch 1985/2000
712/712 [==============================] - 0s - loss: 0.4518 - acc: 0.7963 - val_loss: 0.4155 - val_acc: 0.8268
Epoch 1986/2000
712/712 [==============================] - 0s - loss: 0.4514 - acc: 0.7963 - val_loss: 0.4150 - val_acc: 0.8268
Epoch 1987/2000
712/712 [==============================] - 0s - loss: 0.4512 - acc: 0.7992 - val_loss: 0.4146 - val_acc: 0.8268
Epoch 1988/2000
712/712 [==============================] - 0s - loss: 0.4517 - acc: 0.7949 - val_loss: 0.4151 - val_acc: 0.8268
Epoch 1989/2000
712/712 [==============================] - 0s - loss: 0.4517 - acc: 0.7992 - val_loss: 0.4149 - val_acc: 0.8268
Epoch 1990/2000
712/712 [==============================] - 0s - loss: 0.4513 - acc: 0.7963 - val_loss: 0.4155 - val_acc: 0.8268
Epoch 1991/2000
712/712 [==============================] - 0s - loss: 0.4514 - acc: 0.8034 - val_loss: 0.4164 - val_acc: